# Predicting Athletes' Success based on Physical Built - Iterations

In [1]:
from ibm_botocore.client import Config
import ibm_boto3

In [2]:
SEED = 42

# Numpy
import numpy as np
np.random.seed(SEED)
from numpy import loadtxt

# Pandas
import pandas as pd

# Sklearn

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.model_selection import cross_validate, learning_curve, train_test_split
from sklearn.model_selection import cross_val_score

# Metrics and tuning
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression as LR
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# PySpark
!pip install pyspark==2.4.5
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lit
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Stats
from scipy import stats

# Vizualization
import seaborn as sns
sns.set(style="whitegrid")

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [3]:
from pyspark import SparkContext
sc =SparkContext()

22/01/03 14:28:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Data

In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_ae9960ddc8814d4c8b5407d633c117e6 = 'https://s3.ap.cloud-object-storage.appdomain.cloud'
else:
    endpoint_ae9960ddc8814d4c8b5407d633c117e6 = 'https://s3.private.ap.cloud-object-storage.appdomain.cloud'

client_ae9960ddc8814d4c8b5407d633c117e6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='y-i7Ll-_fH7OmwBZU47jYLDcYfCQFgbdSFfHOig9oGAC',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_ae9960ddc8814d4c8b5407d633c117e6)

body = client_ae9960ddc8814d4c8b5407d633c117e6.get_object(Bucket='ibmadvanceddatasciencecapstone-donotdelete-pr-imkca0ueactk84',Key='train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,0,1,2,3,4,5,6,7,8,9,...,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254
0,-1.651038,-2.027846,-1.511511,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.717441,0.250291,1.908546,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.922275,1.769049,1.140778,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.922275,-0.604011,-0.953134,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.286657,-1.363390,-0.988033,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
body = client_ae9960ddc8814d4c8b5407d633c117e6.get_object(Bucket='ibmadvanceddatasciencecapstone-donotdelete-pr-imkca0ueactk84',Key='test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

,0,1,2,3,4,5,6,7,8,9,...,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254
0,0.170869,-0.319243,-0.743743,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.011322,1.199515,0.582401,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.899632,0.629980,0.512604,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.011322,0.724903,0.373010,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.628395,0.440136,1.070981,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
body = client_ae9960ddc8814d4c8b5407d633c117e6.get_object(Bucket='ibmadvanceddatasciencecapstone-donotdelete-pr-imkca0ueactk84',Key='y_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,Medal
0,0
1,1
2,0
3,0
4,0


In [7]:

body = client_ae9960ddc8814d4c8b5407d633c117e6.get_object(Bucket='ibmadvanceddatasciencecapstone-donotdelete-pr-imkca0ueactk84',Key='y_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_4 = pd.read_csv(body)
df_data_4.head()

,Medal
0,0
1,0
2,1
3,0
4,1


## Normalization

In [8]:
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation

In [9]:
model2 = Sequential()
model2.add(Dense(16, input_dim=1255))
model2.add(BatchNormalization())
model2.add(Activation('relu'))

model2.add(Dense(12, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model2.fit(df_data_1, df_data_3, epochs=10, batch_size=64)


Epoch 1/10
2255/2255 [==============================] - 16s 7ms/step - loss: 0.0000e+00 - accuracy: 0.8528
Epoch 2/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8548
Epoch 3/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8524
Epoch 4/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8532
Epoch 5/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8547
Epoch 6/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8542
Epoch 7/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8534
Epoch 8/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8552
Epoch 9/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8555
Epoch 10/10
2255/2255 [==============

In [10]:

model2.evaluate(df_data_2, df_data_4)

1933/1933 [==============================] - 3s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8536


[0.0, 0.853613555431366]

## Droupout

In [8]:

from tensorflow.keras.layers import Dropout

model3 = Sequential()
model3.add(Dense(16, input_dim=1255, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(12, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model3.fit(df_data_1, df_data_3, epochs=10, batch_size=64)



Epoch 1/10
2255/2255 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8424
Epoch 2/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8527
Epoch 3/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8543
Epoch 4/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8541
Epoch 5/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8529
Epoch 6/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8545
Epoch 7/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8522
Epoch 8/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8541
Epoch 9/10
2255/2255 [==============================] - 13s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8528
Epoch 10/10
2255/2255 [==============

In [9]:
model3.evaluate(df_data_2, df_data_4)

1933/1933 [==============================] - 4s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8536


[0.0, 0.853613555431366]